# Optuna

In [1]:
%pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
            'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-19 17:48:39,411] A new study created in memory with name: no-name-473f5c5a-ad06-4f22-aa24-cfdd64344b8a
[I 2025-09-19 17:48:40,112] Trial 0 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 104, 'max_depth': 3}. Best is trial 0 with value: 0.7560521415270017.
[I 2025-09-19 17:48:41,566] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 200, 'max_depth': 17}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-19 17:48:42,216] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 85, 'max_depth': 19}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-19 17:48:43,115] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 137, 'max_depth': 6}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-09-19 17:48:44,195] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 157, 'max_depth': 18}. Best is trial 1 with value: 0.7746741

In [7]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 20}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-09-19 17:49:18,604] A new study created in memory with name: no-name-051e96f9-73cc-4177-aa27-af92de2b55ae
[I 2025-09-19 17:49:19,882] Trial 0 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 191, 'max_depth': 20}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-19 17:49:20,306] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 62, 'max_depth': 14}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-19 17:49:21,333] Trial 2 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 156, 'max_depth': 9}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-19 17:49:22,538] Trial 3 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 191, 'max_depth': 6}. Best is trial 0 with value: 0.7709497206703911.
[I 2025-09-19 17:49:23,825] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 200, 'max_depth': 17}. Best is trial 4 with value: 0.7746741

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7783985102420857
Best hyperparameters: {'n_estimators': 177, 'max_depth': 16}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-09-19 17:49:59,622] A new study created in memory with name: no-name-9b898ddd-1445-452b-bb31-011020b94e35
[I 2025-09-19 17:50:00,253] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-19 17:50:01,235] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-09-19 17:50:01,582] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-19 17:50:02,251] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-09-19 17:50:02,956] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [17]:
import matplotlib.pyplot as plt

## Optuna Visualizations

In [18]:
# For visualizations
import matplotlib.pyplot as plt
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances
# Show Optuna optimization history plot
plot_optimization_history(study)
plt.show()

In [19]:
# 1. Optimization History
plot_optimization_history(study).show()

In [20]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [21]:
# 3. Slice Plot
plot_slice(study).show()

In [22]:
# 4. Contour Plot
plot_contour(study).show()

In [23]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [24]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [25]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [26]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-09-19 17:50:15,260] A new study created in memory with name: no-name-d9f5e0c0-c718-4f30-ac50-79a405d2df65
[I 2025-09-19 17:50:15,333] Trial 0 finished with value: 0.7094972067039106 and parameters: {'classifier': 'SVM', 'C': 10.742680208433027, 'kernel': 'poly', 'gamma': 'scale'}. Best is trial 0 with value: 0.7094972067039106.
[I 2025-09-19 17:50:19,738] Trial 1 finished with value: 0.7467411545623835 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 200, 'learning_rate': 0.05855703509770304, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.7467411545623835.
[I 2025-09-19 17:50:21,524] Trial 2 finished with value: 0.7579143389199254 and parameters: {'classifier': 'RandomForest', 'n_estimators': 293, 'max_depth': 11, 'min_samples_split': 9, 'min_samples_leaf': 7, 'bootstrap': True}. Best is trial 2 with value: 0.7579143389199254.
[I 2025-09-19 17:50:22,458] Trial 3 finished with value: 0.7504655493482307 and paramet

In [27]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.11174035581175237, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [28]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.709497,2025-09-19 17:50:15.263207,2025-09-19 17:50:15.332112,0 days 00:00:00.068905,10.742680,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.746741,2025-09-19 17:50:15.333112,2025-09-19 17:50:19.738065,0 days 00:00:04.404953,NaN,NaN,GradientBoosting,NaN,NaN,0.058557,15.0,2.0,9.0,200.0,COMPLETE
2,2,0.757914,2025-09-19 17:50:19.739062,2025-09-19 17:50:21.524029,0 days 00:00:01.784967,NaN,True,RandomForest,NaN,NaN,NaN,11.0,7.0,9.0,293.0,COMPLETE
3,3,0.750466,2025-09-19 17:50:21.525035,2025-09-19 17:50:22.458103,0 days 00:00:00.933068,NaN,NaN,GradientBoosting,NaN,NaN,0.292410,8.0,7.0,5.0,78.0,COMPLETE
4,4,0.756052,2025-09-19 17:50:22.459105,2025-09-19 17:50:26.029649,0 days 00:00:03.570544,NaN,NaN,GradientBoosting,NaN,NaN,0.041502,9.0,7.0,10.0,286.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2025-09-19 17:51:01.424919,2025-09-19 17:51:01.458907,0 days 00:00:00.033988,0.103446,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-09-19 17:51:01.459928,2025-09-19 17:51:01.498118,0 days 00:00:00.038190,0.331739,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-09-19 17:51:01.499118,2025-09-19 17:51:01.532544,0 days 00:00:00.033426,0.195167,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.750466,2025-09-19 17:51:01.533543,2025-09-19 17:51:04.964224,0 days 00:00:03.430681,NaN,NaN,GradientBoosting,NaN,NaN,0.022301,11.0,8.0,9.0,272.0,COMPLETE


In [29]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 71
RandomForest        17
GradientBoosting    12
Name: count, dtype: int64

In [30]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.744879
RandomForest        0.765363
SVM                 0.776589
Name: value, dtype: float64